In [1]:
!pip install -U datasets

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

In [2]:
from datasets import load_dataset

#  Correct approach
dataset = load_dataset(
    "roneneldan/TinyStories",
    split="train"  # Use "default" config implicitly
)

# Verify the dataset
print(dataset[0]["text"])  # Should output a story

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.

Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."

Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.


In [3]:
from google.colab import drive  # Required in every new session
from datasets import load_from_disk

drive.mount('/content/drive')  # Re-authenticate (tiny bandwidth)
dataset = load_from_disk("/content/drive/MyDrive/TinyStories")  # No internet needed

Mounted at /content/drive


In [4]:
# 1. Mount Drive again (NEW SESSION)
from google.colab import drive
drive.mount('/content/drive')

# 2. Load dataset from Drive
from datasets import load_from_disk
dataset = load_from_disk("/content/drive/MyDrive/TinyStories")

# 3. Print the SAME story (NO INTERNET REQUIRED)
print("After restarting session:")
print(dataset[0]["text"][:100] + "...")  # Same partial text

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
After restarting session:
One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with...


In [5]:
# Required Libraries
import torch
import torch.nn as nn
from torch.nn import functional as F
from datasets import load_from_disk
from google.colab import drive
import numpy as np
import os
from glob import glob

In [6]:
# Hyperparameters
batch_size = 16
block_size = 32
max_iters = 10000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

cpu


In [7]:
# Set random seed
torch.manual_seed(1337)
# 2. Load only 75% of the dataset
n = int(0.75 * len(dataset))
dataset = dataset.select(range(n))

In [8]:
# 3. Concatenate and Tokenize
text = "".join(dataset[i]["text"] for i in range(len(dataset)))
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [9]:
# 4. Encode and save in chunks
output_dir = "/content/encoded_chunks"
os.makedirs(output_dir, exist_ok=True)

chunk_size = 1000
counter = 0
for i in range(0, len(dataset), chunk_size):
    chunk = [dataset[j]["text"] for j in range(i, min(i + chunk_size, len(dataset)))]
    encoded = [encode(text) for text in chunk]
    flat = [item for sublist in encoded for item in sublist]
    chunk_tensor = torch.tensor(flat, dtype=torch.int32)
    torch.save(chunk_tensor, os.path.join(output_dir, f"chunk_{counter}.pt"))
    counter += 1

In [10]:
# 5. Combine into single binary file
final_output_path = "/content/full_encoded_data.bin"
with open(final_output_path, 'wb') as f:
    chunk_paths = sorted(glob(os.path.join(output_dir, "chunk_*.pt")))
    total_tokens = 0
    for path in chunk_paths:
        chunk = torch.load(path)
        f.write(chunk.numpy().astype('int32').tobytes())
        total_tokens += chunk.numel()

In [11]:
# 6. Split into train/val
split_point = int(0.9 * total_tokens)
train_path = "/content/train_data.bin"
val_path = "/content/val_data.bin"
with open(final_output_path, 'rb') as f:
    train_bytes = f.read(split_point * 4)
    with open(train_path, 'wb') as train_file:
        train_file.write(train_bytes)
    val_bytes = f.read()
    with open(val_path, 'wb') as val_file:
        val_file.write(val_bytes)

In [12]:
# 7. Memmap train/val
train_data = np.memmap(train_path, dtype=np.int32, mode='r')
val_data = np.memmap(val_path, dtype=np.int32, mode='r')

# 8. Data loader

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([torch.tensor(data[i:i+block_size], dtype=torch.long) for i in ix])
    y = torch.stack([torch.tensor(data[i+1:i+block_size+1], dtype=torch.long) for i in ix])
    return x.to(device), y.to(device)


In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
# 9. Model definition
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

In [15]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(out))

In [16]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [17]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [18]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            logits = logits.view(B*T, -1)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [19]:
# 10. Instantiate model
model = BigramLanguageModel().to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# 11. Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

0.223145 M parameters
step 0: train loss 5.2577, val loss 5.2626
step 100: train loss 2.5419, val loss 2.5393
step 200: train loss 2.3581, val loss 2.3554
step 300: train loss 2.2157, val loss 2.2122
step 400: train loss 2.1300, val loss 2.1413
step 500: train loss 2.0874, val loss 2.0713
step 600: train loss 1.9904, val loss 1.9936
step 700: train loss 1.9360, val loss 1.9273
step 800: train loss 1.8755, val loss 1.8780
step 900: train loss 1.8152, val loss 1.8220
step 1000: train loss 1.7739, val loss 1.7722
step 1100: train loss 1.7369, val loss 1.7313
step 1200: train loss 1.7066, val loss 1.7032
step 1300: train loss 1.6844, val loss 1.6832
step 1400: train loss 1.6615, val loss 1.6601
step 1500: train loss 1.6206, val loss 1.6223
step 1600: train loss 1.5997, val loss 1.6038
step 1700: train loss 1.5920, val loss 1.5854
step 1800: train loss 1.5783, val loss 1.5812
step 1900: train loss 1.5610, val loss 1.5676
step 2000: train loss 1.5425, val loss 1.5488
step 2100: train loss 1.

In [20]:
# 12. Generate text
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=5000)[0].tolist()))

	let oway.Mom said the comes off treeze.""

On. Lily didn't very having.Lily been?"

Her smiled and sece creat it's chate for Lily. They everyone that day, she the veasitly with he couldn't want to not things. The rain named Lily.

When you have to threw are renched, notwies watchester.

One day, he had a little britch found silong.

The little girl named Lucy laugh oversed out he the next for sweet".Sara was scared agail wagily. She ammazed and said the big, and went to the old carebratuat!" The triestingere the little bird who drived and take she a servone. She racted to shape. She circlents okay, from old pretty! Mommy scelfed seen happy. Sam so had she peeced in the reusurplach happy. She had so much as scare?"


Jack fencrous it handed and crastun felt better. Everywhere she felt un was very for or playing with Sara. So You showed her sit finisheds.
She liked to when the best many. She tri€™ted were relievs deady! Everyoney, but streezed and anumages.Once upon a time, there was ap